In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import os
import cv2 as cv
from PIL import Image , ImageOps
import csv
import re
import random
from resizeimage import resizeimage


In [2]:
####  DATA PREP ##################################

directory = os.listdir('data/') #saves all file names to array
random.shuffle(directory) #randomize / shuffle elements

f = []
y = []
df = np.zeros((154,1600))


for im in directory:
    
    if(im[0] != '.'):
        path = 'data/'+im 
        with Image.open(path) as image:
            resized = resizeimage.resize_cover(image, [40, 40])
            resized.save('resized', image.format) 
        y.append(int(re.findall(r'\d+', im)[0])-2) #extract subject num to assign as target class
        f.append(np.array(resized))

d = np.array(f)
d = np.divide(d, 255) #division by 255 to keep vals in range (0,1

#flatten observations into rows of 1x1600
for i in range(154):
    df[i] = np.ndarray.flatten(d[i])

#truncate vals to remain in range (0,1)
df[df<0] = 0
df[df>1] = 1

#one hot encoding matrix creation - target class labels
one_hot = np.zeros((np.shape(d)[0],len(np.unique(y))))
i = 0

for c in y:
    one_hot[i][c] = 1
    i+=1

#split into train set (103x1600) & validation set (51x1600)
x_train = df[:103:]
y_train = one_hot[:103:]
x_test = df[-51:]
y_test = one_hot[-51:]

print("train set:\n   x", np.shape(x_train), "\n   y", np.shape(y_train ))
print("test set:\n   x", np.shape(x_test), "\n   y", np.shape(y_test))




train set:
   x (103, 1600) 
   y (103, 14)
test set:
   x (51, 1600) 
   y (51, 14)


In [ ]:
####  MULTI-CLASS CLASSIFIER ###########################
# INPUT -> FC -> ACT (SIGMOID) -> OUT -> CROSS ENTROPY ACTIVATION

def fc_layer(x, w, b):
    '''Input -> 103 obs x 1600 features
       w     -> 14 classes x 1600 features
       b     -> 14 classes
       
       outputs 103 obs x 14 classes'''
    
    tmp = []

    for i in range(np.shape(x)[0]):
        tmp.append(np.sum(x[i]*w+b, axis=1))
    
    return np.array(tmp)
    

def softmax(z):
    
    '''output valid probability dist'''
    
    tmp = []

    #for ea observation
    for i in range(np.shape(z)[0]):
        tmp.append(np.exp(z[i] - np.max(z[i])) / ( np.sum(np.exp(z[i] - np.max(z[i]))) ))
    
    return np.array(tmp)
    
def compute_cost(y, yhat):
    '''Cross Entropy
       Input: y - target - one hot encoded - (103x14) 
              yhat - valid dist - (103x14)'''
    tmp = []

    #for ea observation
    for i in range(len(y)):
        tmp.append( -y[i] * np.log(yhat[i]) )
    
    return np.array(tmp)

def compute_gradients(x, y, yhat):
    
    '''Gradient of softmax w cross entropy'''
    
    n=len(y)
    
    g_w = x_train.T@(yhat - y)
    g_b = np.sum((yhat - y), axis=0)
    
    return g_w, g_b.reshape((14,1))

def update_weights(w, b, g_w, g_b, lr):
    
    '''Increase gradient w restult from compute_gradient() to weights'''
    
    tmp_w = []
    tmp_b = []

    #for ea class
    for i in range(14):
        tmp_w.append( np.add(w[i],-(lr*g_w[:,i])) )
        tmp_b.append( np.add(b[i],-(lr*g_b[i])) )

    return np.array(tmp_w), np.array(tmp_b)   

def terminate(j_p, j_c):
    
    return np.round(np.average(j_p), 4) == np.round(np.average(j_c), 4)

def classify(y_hat, y):

    '''Cutoff is 0.5 - any values that fall above are set to 1 and any below to 0'''
    
    classified = []
    
    for i in range(len(yhat)):

        classified.append(np.argmax(yhat))
    
    return classified

def accuracy(y_hat, y):
    
    tp = 0
    err = 0

    #for ea observation
    for i in range(len(y)):

        #if the highest prob class in yhat matches the target class
        if( np.argmax(y[i]) == np.argmax(y_hat[i]) ):
            tp += 1
        else:
            err += 1
    
    return tp, err

def visualize(epochs, j):
    
    '''Charts J vs Epoch'''

    epoch=np.linspace(1,epochs,num=epochs)

    print("epoch vs J ->")
    fig, axes = plt.subplots()
    axes.plot(epoch, j)
    axes.set_xlabel('epoch')
    axes.set_ylabel('J')
    plt.show()

#####################################################

#init weights
w = np.random.uniform(-10**-4,10**-4, (14,1600))
b = np.random.uniform(-10**-4,10**-4, (14,1))

j_track = [] #tracks chng in cost function - sanity check
l_rate = 10**-4
j=0
epochs = 0

#GRADIENT DESCENT
while(epochs < 1500):
    
    #FC LAYER - composed of fc + activation
    z = fc_layer(x_train, w, b)
    yhat = softmax(z)

    #OUTPUT LAYER - evaluate cost function
    j_prev = j #to track/plot loss function
    j = compute_cost(y_train, yhat) 
    j_track.append(np.average(j))

    #Update Weights
    g_w, g_b = compute_gradients(x_train, y_train, yhat)
    w, b = update_weights(w, b, g_w, g_b, l_rate)
    
    epochs += 1 
#    if(terminate(j_prev,j)): break

########################################################################

#classify / compute accuracy
ztest = fc_layer(x_test, w, b)
yhat_test = softmax(ztest)
tp, err = accuracy(yhat_test, y_test)
print("Accuracy: ", (tp / (tp+err)))


#Visualize epoch V J
epoch=np.linspace(1,epochs,num=epochs)

print("epoch vs J ->")
fig, axes = plt.subplots()
axes.plot(epoch, j_track)
axes.set_xlabel('epoch')
axes.set_ylabel('J')
plt.show()
